In [26]:
import requests
import json

BASE_URL = "http://localhost:8000"

In [27]:
def test_health():
    print("Testing /health endpoint...")
    
    response = requests.get(f"{BASE_URL}/health")
    print(f"Status Code: {response.status_code}")
    print(f"Response: {json.dumps(response.json(), indent=2)}")
    
    assert response.status_code == 200
    print(" Health check passed!")

test_health()

Testing /health endpoint...
Status Code: 200
Response: {
  "status": "healthy",
  "service": "house-price-prediction"
}
 Health check passed!


In [28]:
def test_valid_prediction():
    print("Testing /predict with VALID data...")
    
    data = {
        "OverallQual": 7,
        "GrLivArea": 1500,
        "1stFlrSF": 1000,
        "2ndFlrSF": 500,
        "FullBath": 2,
        "TotRmsAbvGrd": 7,
        "YearBuilt": 2000,
        "LotArea": 8000,
        "KitchenQual": "Gd",
        "Foundation": "PConc",
        "ExterQual": "TA",
        "Neighborhood": "CollgCr"
    }
    
    print("Request data:")
    print(json.dumps(data, indent=2))
    
    response = requests.post(f"{BASE_URL}/predict", json=data)
    print(f"\nStatus Code: {response.status_code}")
    print(f"Response: {json.dumps(response.json(), indent=2)}")
    
    assert response.status_code == 200
    assert "predicted_price" in response.json()
    print(" Valid prediction passed!")

test_valid_prediction()

Testing /predict with VALID data...
Request data:
{
  "OverallQual": 7,
  "GrLivArea": 1500,
  "1stFlrSF": 1000,
  "2ndFlrSF": 500,
  "FullBath": 2,
  "TotRmsAbvGrd": 7,
  "YearBuilt": 2000,
  "LotArea": 8000,
  "KitchenQual": "Gd",
  "Foundation": "PConc",
  "ExterQual": "TA",
  "Neighborhood": "CollgCr"
}

Status Code: 200
Response: {
  "predicted_price": 184311.84375
}
 Valid prediction passed!


In [29]:
def test_with_optional_fields():
    print("Testing /predict with OPTIONAL fields...")
    
    data = {
        "OverallQual": 8,
        "GrLivArea": 2000,
        "1stFlrSF": 1200,
        "2ndFlrSF": 800,
        "FullBath": 2,
        "HalfBath": 1,
        "TotRmsAbvGrd": 9,
        "YearBuilt": 2010,
        "YearRemodAdd": 2015,
        "LotArea": 10000,
        "KitchenQual": "Ex",
        "Foundation": "PConc",
        "ExterQual": "Gd",
        "Neighborhood": "NridgHt",
        "GarageCars": 2,
        "GarageArea": 500,
        "GarageYrBlt": 2010,
        "Fireplaces": 1,
        "TotalBsmtSF": 1200
    }
    
    print("Request data:")
    print(json.dumps(data, indent=2))
    
    response = requests.post(f"{BASE_URL}/predict", json=data)
    print(f"\nStatus Code: {response.status_code}")
    print(f"Response: {json.dumps(response.json(), indent=2)}")
    
    assert response.status_code == 200
    print("Prediction with optional fields passed!")

test_with_optional_fields()

Testing /predict with OPTIONAL fields...
Request data:
{
  "OverallQual": 8,
  "GrLivArea": 2000,
  "1stFlrSF": 1200,
  "2ndFlrSF": 800,
  "FullBath": 2,
  "HalfBath": 1,
  "TotRmsAbvGrd": 9,
  "YearBuilt": 2010,
  "YearRemodAdd": 2015,
  "LotArea": 10000,
  "KitchenQual": "Ex",
  "Foundation": "PConc",
  "ExterQual": "Gd",
  "Neighborhood": "NridgHt",
  "GarageCars": 2,
  "GarageArea": 500,
  "GarageYrBlt": 2010,
  "Fireplaces": 1,
  "TotalBsmtSF": 1200
}

Status Code: 200
Response: {
  "predicted_price": 270033.78125
}
Prediction with optional fields passed!


In [30]:
def test_structural_validation_error():
    print("Testing STRUCTURAL validation error...")
    
    data = {
        "OverallQual": 15,
        "GrLivArea": 1500,
        "1stFlrSF": 1000,
        "FullBath": 2,
        "TotRmsAbvGrd": 7,
        "YearBuilt": 2000,
        "LotArea": 8000,
        "KitchenQual": "Gd",
        "Foundation": "PConc",
        "ExterQual": "TA",
        "Neighborhood": "CollgCr"
    }
    
    print("Request data (OverallQual=15, should be 1-10):")
    print(json.dumps(data, indent=2))
    
    response = requests.post(f"{BASE_URL}/predict", json=data)
    print(f"\nStatus Code: {response.status_code}")
    print(f"Error: {json.dumps(response.json(), indent=2)}")
    
    assert response.status_code == 422  
    print("Structural validation working correctly!")

test_structural_validation_error()

Testing STRUCTURAL validation error...
Request data (OverallQual=15, should be 1-10):
{
  "OverallQual": 15,
  "GrLivArea": 1500,
  "1stFlrSF": 1000,
  "FullBath": 2,
  "TotRmsAbvGrd": 7,
  "YearBuilt": 2000,
  "LotArea": 8000,
  "KitchenQual": "Gd",
  "Foundation": "PConc",
  "ExterQual": "TA",
  "Neighborhood": "CollgCr"
}

Status Code: 422
Error: {
  "detail": [
    {
      "type": "less_than_equal",
      "loc": [
        "body",
        "OverallQual"
      ],
      "msg": "Input should be less than or equal to 10",
      "input": 15,
      "ctx": {
        "le": 10
      },
      "url": "https://errors.pydantic.dev/2.5/v/less_than_equal"
    }
  ]
}
Structural validation working correctly!


In [31]:
def test_semantic_validation_error():
    print("Testing SEMANTIC validation error...")

    data = {
        "OverallQual": 7,
        "GrLivArea": 1500,
        "1stFlrSF": 1000,
        "2ndFlrSF": 500,
        "FullBath": 2,
        "TotRmsAbvGrd": 7,
        "YearBuilt": 2000,
        "YearRemodAdd": 1990,  
        "LotArea": 8000,
        "KitchenQual": "Gd",
        "Foundation": "PConc",
        "ExterQual": "TA",
        "Neighborhood": "CollgCr"
    }
    
    print("Request data (YearRemodAdd=1990, YearBuilt=2000):")
    print(json.dumps(data, indent=2))
    
    response = requests.post(f"{BASE_URL}/predict", json=data)
    print(f"\nStatus Code: {response.status_code}")
    print(f"Error: {json.dumps(response.json(), indent=2)}")
    
    assert response.status_code == 422
    print("Semantic validation working correctly!")

test_semantic_validation_error()

Testing SEMANTIC validation error...
Request data (YearRemodAdd=1990, YearBuilt=2000):
{
  "OverallQual": 7,
  "GrLivArea": 1500,
  "1stFlrSF": 1000,
  "2ndFlrSF": 500,
  "FullBath": 2,
  "TotRmsAbvGrd": 7,
  "YearBuilt": 2000,
  "YearRemodAdd": 1990,
  "LotArea": 8000,
  "KitchenQual": "Gd",
  "Foundation": "PConc",
  "ExterQual": "TA",
  "Neighborhood": "CollgCr"
}

Status Code: 422
Error: {
  "detail": [
    {
      "type": "value_error",
      "loc": [
        "body",
        "YearRemodAdd"
      ],
      "msg": "Value error, YearRemodAdd (1990) cannot be before YearBuilt (2000)",
      "input": 1990,
      "ctx": {
        "error": {}
      },
      "url": "https://errors.pydantic.dev/2.5/v/value_error"
    }
  ]
}
Semantic validation working correctly!


In [32]:
def test_logical_validation_error():
    print("Testing LOGICAL validation error...")
    
    data = {
        "OverallQual": 7,
        "GrLivArea": 1500,
        "1stFlrSF": 1000,
        "2ndFlrSF": 500,
        "FullBath": 2,
        "TotRmsAbvGrd": 7,
        "YearBuilt": 2000,
        "LotArea": 8000,
        "KitchenQual": "Gd",
        "Foundation": "PConc",
        "ExterQual": "TA",
        "Neighborhood": "CollgCr",
        "GarageCars": 2,
        "GarageArea": 0 
    }
    
    print("Request data (GarageCars=2 but GarageArea=0):")
    print(json.dumps(data, indent=2))
    
    response = requests.post(f"{BASE_URL}/predict", json=data)
    print(f"\nStatus Code: {response.status_code}")
    print(f"Error: {json.dumps(response.json(), indent=2)}")
    
    assert response.status_code == 422
    print("Logical validation working correctly!")

test_logical_validation_error()

Testing LOGICAL validation error...
Request data (GarageCars=2 but GarageArea=0):
{
  "OverallQual": 7,
  "GrLivArea": 1500,
  "1stFlrSF": 1000,
  "2ndFlrSF": 500,
  "FullBath": 2,
  "TotRmsAbvGrd": 7,
  "YearBuilt": 2000,
  "LotArea": 8000,
  "KitchenQual": "Gd",
  "Foundation": "PConc",
  "ExterQual": "TA",
  "Neighborhood": "CollgCr",
  "GarageCars": 2,
  "GarageArea": 0
}

Status Code: 422
Error: {
  "detail": [
    {
      "type": "value_error",
      "loc": [
        "body"
      ],
      "msg": "Value error, Cannot have GarageCars (2) without GarageArea",
      "input": {
        "OverallQual": 7,
        "GrLivArea": 1500,
        "1stFlrSF": 1000,
        "2ndFlrSF": 500,
        "FullBath": 2,
        "TotRmsAbvGrd": 7,
        "YearBuilt": 2000,
        "LotArea": 8000,
        "KitchenQual": "Gd",
        "Foundation": "PConc",
        "ExterQual": "TA",
        "Neighborhood": "CollgCr",
        "GarageCars": 2,
        "GarageArea": 0
      },
      "ctx": {
        "erro

In [33]:
def test_floor_areas_validation():
    print("Testing FLOOR AREAS validation...")
    
    data = {
        "OverallQual": 7,
        "GrLivArea": 1500,
        "1stFlrSF": 1000,
        "2ndFlrSF": 200, 
        "FullBath": 2,
        "TotRmsAbvGrd": 7,
        "YearBuilt": 2000,
        "LotArea": 8000,
        "KitchenQual": "Gd",
        "Foundation": "PConc",
        "ExterQual": "TA",
        "Neighborhood": "CollgCr"
    }
    
    print("Request data (1stFlr=1000 + 2ndFlr=200 ≠ GrLivArea=1500):")
    print(json.dumps(data, indent=2))
    
    response = requests.post(f"{BASE_URL}/predict", json=data)
    print(f"\nStatus Code: {response.status_code}")
    print(f"Error: {json.dumps(response.json(), indent=2)}")
    
    assert response.status_code == 422
    print("Floor areas validation working correctly!")

test_floor_areas_validation()

Testing FLOOR AREAS validation...
Request data (1stFlr=1000 + 2ndFlr=200 ≠ GrLivArea=1500):
{
  "OverallQual": 7,
  "GrLivArea": 1500,
  "1stFlrSF": 1000,
  "2ndFlrSF": 200,
  "FullBath": 2,
  "TotRmsAbvGrd": 7,
  "YearBuilt": 2000,
  "LotArea": 8000,
  "KitchenQual": "Gd",
  "Foundation": "PConc",
  "ExterQual": "TA",
  "Neighborhood": "CollgCr"
}

Status Code: 422
Error: {
  "detail": [
    {
      "type": "value_error",
      "loc": [
        "body"
      ],
      "msg": "Value error, Floor areas don't match: 1stFlr (1000) + 2ndFlr (200) = 1200 but GrLivArea is 1500",
      "input": {
        "OverallQual": 7,
        "GrLivArea": 1500,
        "1stFlrSF": 1000,
        "2ndFlrSF": 200,
        "FullBath": 2,
        "TotRmsAbvGrd": 7,
        "YearBuilt": 2000,
        "LotArea": 8000,
        "KitchenQual": "Gd",
        "Foundation": "PConc",
        "ExterQual": "TA",
        "Neighborhood": "CollgCr"
      },
      "ctx": {
        "error": {}
      },
      "url": "https://err